In [ ]:
import sys
sys.path.append('../')

from model import *
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# MIMIC - Triage

This notebook runs the analysis on the MIMIC emergency data by leveraging experts' agreement
1. Explore a model build on data ignoring experts 
2. Compute agreement between experts using influence function
3. Retrain the model on the set of labels for which experts strongly agree

The current analysis uses multi layer perceptrons in a single train / test split.

### Data

Reopen the data created with the notebook in `data/`

To use with your data, you will need to change the following line to open a file with:
- `X` covariates
- `H` associated (human) experts 
- `D` their decision for each case
- `Y` observed outcome
- `Yc` a concept 

In [ ]:
triage = pd.read_csv('../data/triage_clean.csv', index_col = [0, 1])
covariates, target, nurses = triage.drop(columns = ['D', 'Y1', 'Y2', 'YC', 'acuity', 'nurse']), triage[['D', 'Y1', 'Y2', 'YC']], triage['nurse']

Split data in a 80% train, 20% test

In [ ]:
cov_train, cov_test, tar_train, tar_test, nur_train, nur_test = train_test_split(covariates, target, nurses, test_size = 0.2, random_state = 42)

### Modelling

In [ ]:
# Model's characteristics
params = {'layers': []} # If = [] equivalent to a simple logistic regression

# Amalgation parameters
rho = 0.05 # Control which point to consider from a confience point of view
pi_1 = 4.0 # Control criterion on centre mass metric
pi_2 = 0.8 # Control criterion on opposing metric
tau = 1.0  # Balance between observed and expert labels

##### 1. Train on decision

This model models the nurse decision based on covariates

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['D'], nur_train)

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'], model.predict(cov_test))

In [ ]:
# D performance
roc_auc_score(tar_test['D'], model.predict(cov_test))

##### 2. Agreement computation 

Measure of agreeability are estimated in a cross validation fashion on the train set.

In [ ]:
# Fold evaluation of influences
folds, predictions, influence = influence_cv(BinaryMLP, cov_train, tar_train['D'], nur_train, params = params)

In [ ]:
# Compute metrics agreeability
center_metric, opposing_metric = compute_agreeability(influence)

In [ ]:
# Apply criteria on amalgamation
high_conf = (predictions > (1 - rho)) | (predictions < rho)
high_agr = (center_metric > pi_1) & (opposing_metric > pi_2) & high_conf
high_agr_correct = ((predictions - tar_train['D']).abs() < rho) & high_agr

In [ ]:
# Create amalgamated labels
tar_train['Ya'] = tar_train['Y1']
tar_train['Ya'][high_agr_correct] = (1 - tau) * tar_train['Y1'][high_agr_correct] \
                                    + tau * tar_train['D'][high_agr_correct]

In [ ]:
index_amalg = tar_train['D'] | high_agr_correct

##### 3. Updated model

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train[index_amalg], tar_train[index_amalg]['Ya'], nur_train[index_amalg])

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'],model.predict(cov_test))

In [ ]:
# D performance
roc_auc_score(tar_test['D'], model.predict(cov_test))

##### 4. Train on observed data

In [ ]:
model = BinaryMLP(**params)
model = model.fit(cov_train, tar_train['Y1'], nur_train)

In [ ]:
# Naive performance
roc_auc_score(tar_test['Y1'], model.predict(cov_test))

In [ ]:
# Yc performance
roc_auc_score(tar_test['YC'],model.predict(cov_test))

In [ ]:
# D performance
roc_auc_score(tar_test['D'], model.predict(cov_test))